# Spopularity

---


# Requesting new token

First request is for updating the token, if needed. Credentials are stored in `credentials.json` -file.


In [1]:
import json
import spotipy
import time


def get_token():
    client_id = ""
    client_secret = ""
    token = ""
    expiration = 0

    with open("credentials.json", "r") as f:
        credentials = json.load(f)
        client_id = credentials["client_id"]
        client_secret = credentials["client_secret"]
        token = credentials["token"]
        expiration = credentials["expiration"]

    if not token or expiration < time.time():
        token = spotipy.oauth2.SpotifyClientCredentials(
            client_id=client_id, client_secret=client_secret
        )
        token = token.get_access_token(as_dict=False)
        credentials["token"] = token
        credentials["expiration"] = int(time.time() + 3600)

        with open("credentials.json", "w") as f:
            f.write(json.dumps(credentials))

    return token


s = spotipy.Spotify(get_token())

# Requests to RapidAPI Billboard top 100 weekly chart

**Not needed for requesting track data from Spotify**

Requires registering an account. **Limit for monthly requests is 30 (use wisely).**

Headers (credentials) also in `credentials.json`.


In [2]:
import requests

url = "https://billboard-api2.p.rapidapi.com/hot-100"
querystring = {"date":"2023-05-11","range":"1-100"}

file = open("credentials.json", "r")
credentials = json.load(file)
rapid_api_key, rapid_api_host = credentials["rapid-api-key"], credentials["rapid-api-host"]
file.close()

headers = {
    "X-RapidAPI-Key": rapid_api_key,
    "X-RapidAPI-Host": rapid_api_host
}

# response = requests.get(url, headers=headers, params=querystring)

# print(response.json())

In [3]:
import pandas as pd


billboard_top_5 = [
    "1kuGVB7EU95pJObxwvfwKS",
    "2IGMVunIBsBLtEQyoI1Mu7",
    "4KULAymBBJcPRpk1yO4dOG",
    "1Lo0QY9cvc8sUB2vnIOxDT",
    "1BxfuPKGuaTgP7aM0Bbdwr",
]

track_details = s.tracks(billboard_top_5)

tracks = []
for t in track_details["tracks"]:
    audio_features = s.audio_features(t["id"])
    track_name = t["name"]
    track_id = t["id"]
    artist_name = t["artists"][0]["name"]
    popularity = t["popularity"]
    
    audio_features = s.audio_features(t["id"])[0]    
    danceability = audio_features["danceability"]
    energy = audio_features["energy"]
    key = audio_features["key"]
    loudness = audio_features["loudness"]
    mode = audio_features["mode"]
    speechiness = audio_features["speechiness"]
    acousticness = audio_features["acousticness"]
    instrumentalness = audio_features["instrumentalness"]
    liveness = audio_features["liveness"]
    valence = audio_features["valence"]
    tempo = audio_features["tempo"]
    type = audio_features["type"]
    id = audio_features["id"]
    uri = audio_features["uri"]
    track_href = audio_features["track_href"]
    analysis_url = audio_features["analysis_url"]
    duration_ms = audio_features["duration_ms"]
    time_signature = audio_features["time_signature"]
    
    # audio_analysis = s.audio_analysis(t["id"])
    
    tracks.append([track_name, track_id, artist_name, popularity, danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature])

df = pd.DataFrame(tracks)

print(df.head())
print(df.describe())

                                              0                       1   \
0                                        vampire  1kuGVB7EU95pJObxwvfwKS   
1                             Paint The Town Red  2IGMVunIBsBLtEQyoI1Mu7   
2  I Remember Everything (feat. Kacey Musgraves)  4KULAymBBJcPRpk1yO4dOG   
3                                       Fast Car  1Lo0QY9cvc8sUB2vnIOxDT   
4                                   Cruel Summer  1BxfuPKGuaTgP7aM0Bbdwr   

               2    3      4      5   6      7   8       9   ...      12  \
0  Olivia Rodrigo   93  0.511  0.532   5 -5.745   1  0.0578  ...  0.2910   
1        Doja Cat  100  0.868  0.538   5 -8.603   1  0.1740  ...  0.0901   
2      Zach Bryan   92  0.429  0.453   0 -7.746   1  0.0459  ...  0.1020   
3      Luke Combs   89  0.712  0.603   8 -5.520   1  0.0262  ...  0.1150   
4    Taylor Swift   98  0.552  0.702   9 -5.707   1  0.1570  ...  0.1050   

      13       14              15                      16  \
0  0.350  138.005  audio_